In [1]:
import torch.nn as nn
import torch

In [3]:
index = 1
while True:
    a = nn.Linear(10000,100*index)
    a.cuda(1)
    index +=1 

In [4]:
linear = nn.Linear(2,4)

In [10]:
linear.weight

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)

In [9]:
nn.init.zeros_(linear.weight)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)

In [16]:
class MOELoraLayer(nn.Module):
    def __init__(self, dim, r, expert_num, hydra=False):
        super().__init__()
        self.expert_num = expert_num
        self.hydra = hydra # hydra lora

        self.router = nn.Linear(dim, expert_num, bias=False)

        if hydra:
            self.lora_A = nn.Linear(dim, r, bias=False)
        else:
            self.lora_A = nn.ModuleList()
            for i in range(expert_num):
                self.lora_A.append(nn.Linear(dim, r, bias=False))
            
        self.lora_B = nn.ModuleList()
        for i in range(expert_num):
            self.lora_B.append(nn.Linear(r, dim, bias=False))

        # initial lora B to zeros
        for linear in self.lora_B:
            nn.init.zeros_(linear.weight)

    def forward(self, x: torch.Tensor):
        route_weight = nn.functional.softmax(self.router(x), dim=-1, dtype=torch.float32).to(x.dtype)
        # try lora_alpha
        for i in range(self.expert_num):
            if self.hydra:
                x = x + torch.unsqueeze(route_weight[:,:,i], -1) * self.lora_B[i](self.lora_A(x))
            else:
                x = x + torch.unsqueeze(route_weight[:,:,i], -1) * self.lora_B[i](self.lora_A[i](x))
        return x

In [17]:
moelora = MOELoraLayer(10,2,5)

In [29]:
moelora.state_dict().keys()

odict_keys(['router.weight', 'lora_A.0.weight', 'lora_A.1.weight', 'lora_A.2.weight', 'lora_A.3.weight', 'lora_A.4.weight', 'lora_B.0.weight', 'lora_B.1.weight', 'lora_B.2.weight', 'lora_B.3.weight', 'lora_B.4.weight'])

In [24]:
x = torch.randn(3,4)
a = torch.randn(4,2)
a1= torch.randn(4,2)
a2= torch.randn(4,2)

b1 = torch.randn(2,4)
b2= torch.randn(2,4)

relu = nn.ReLU()

In [20]:
(relu(x @ a1) @ b1)*0.4 + (relu(x @ a2) @ b2)*0.6

tensor([[ 0.0131,  0.8114, -0.1674,  0.1921],
        [ 0.3173,  1.5902,  1.2141,  0.6184],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [21]:
relu(x @ a1) @ (b1*0.4) + relu(x @ a2) @ (b2*0.6)

tensor([[ 0.0131,  0.8114, -0.1674,  0.1921],
        [ 0.3173,  1.5902,  1.2141,  0.6184],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [25]:
relu(x @ a) @ ( b1*0.4) + relu(x @ a) @ (b2*0.6)

tensor([[-0.5762, -0.2077, -0.6032, -0.1846],
        [-0.3817, -0.1376, -0.3996, -0.1223],
        [ 0.0645, -0.0288,  0.0293, -0.0455]])

In [26]:
relu(x @ a) @ ( b1*0.4 + b2*0.6)

tensor([[-0.5762, -0.2077, -0.6032, -0.1846],
        [-0.3817, -0.1376, -0.3996, -0.1223],
        [ 0.0645, -0.0288,  0.0293, -0.0455]])

In [2]:
a = [1,2,3,4]
ta = torch.FloatTensor(a)

In [8]:
ta.dtype

torch.float32

In [6]:
nn.functional.sigmoid(ta)

tensor([0.7311, 0.8808, 0.9526, 0.9820])

In [12]:
nn.functional.softmax(ta)*4


/tmp/ipykernel_1876773/2441912604.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(ta)*4


tensor([0.1282, 0.3486, 0.9475, 2.5757])

In [14]:
ta.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4])

In [2]:
a = torch.randn(3,4,2)

In [3]:
b = torch.randn(3,4,2,5)

In [9]:
torch.sum(a.unsqueeze(-1) * b, 2, keepdim=True).shape

torch.Size([3, 4, 1, 5])

In [4]:
a == None

False

# math length

In [4]:
from LLaMA3_lora_bias.llama import Tokenizer
model_path = '/home2/caojie/pretrain_models/Meta-Llama-3-8B/'
tokenizer = Tokenizer(model_path= f"{model_path}/tokenizer.model")

/home2/caojie/miniconda3/envs/summary/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


------ flash attention2 enable -----


In [13]:
import json
# with open(f'/home2/caojie/projects/LLM-Adapters/ft-training_set/math_14k.json', 'r') as f:
# with open(f'/home2/caojie/projects/LLM-Adapters/ft-training_set/commonsense_15k.json', 'r') as f:
with open(f'/home2/caojie/datasets/math_commonsense/hellaswag/test.json', 'r') as f:
    data = f.read()

data = json.loads(data)

In [14]:
data[1]

{'instruction': 'Please choose the correct ending to complete the given sentence: Clean and jerk: A lady walks to a barbell. She bends down and grabs the pole. the lady\n\nEnding1: swings and lands in her arms. Ending2: pulls the barbell forward. Ending3: pulls a rope attached to the barbell. Ending4: stands and lifts the weight over her head.\n\nAnswer format: ending1/ending2/ending3/ending4',
 'input': '',
 'output': 'the correct answer is ending4',
 'answer': 'ending4'}

In [15]:
input_lens=[]
output_lens=[]
for x in data:
    input_lens.append(len(tokenizer.encode(x['instruction']+x['output'], bos=False, eos=False)))
    output_lens.append(len(tokenizer.encode(x['output'], bos=False, eos=False)))
print(f'average tokens:{sum(input_lens)/len(input_lens)}')
print(f'average tokens:{sum(output_lens)/len(output_lens)}')

average tokens:223.45678151762598
average tokens:6.0


In [16]:
max(input_lens)


391

In [17]:
len(input_lens)

10042

In [1]:
a = {'a':1, 'b':2}

In [7]:
sum(a.values())/len(a.values())

1.5

In [1]:
list(range(0,32))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [2]:
a = torch.randn(2,4)

In [4]:
a.unsqueeze(1).shape

torch.Size([2, 1, 4])

In [2]:
a = torch.nn.Parameter(torch.zeros(1))

In [3]:
b = torch.randn(3,4)

In [4]:
a * b

tensor([[-0., -0., 0., -0.],
        [0., 0., -0., -0.],
        [0., 0., -0., -0.]], grad_fn=<MulBackward0>)

In [1]:
a = [1,2,3]

In [2]:
a[1:None]

[2, 3]

In [2]:
import torch

In [9]:
a = torch.nn.Parameter(torch.zeros(3,4).unsqueeze(0))

In [6]:
a.device

device(type='cpu')

In [11]:
torch.nn.init.normal_(a.data)

tensor([[[-1.4849, -0.9344,  1.2820, -0.6792],
         [ 1.1104, -2.3331,  1.0779,  1.6265],
         [-1.2120,  0.3349, -0.3242,  1.1450]]])

In [12]:
a

Parameter containing:
tensor([[[-1.4849, -0.9344,  1.2820, -0.6792],
         [ 1.1104, -2.3331,  1.0779,  1.6265],
         [-1.2120,  0.3349, -0.3242,  1.1450]]], requires_grad=True)

In [1]:
import torch
import torch.nn as nn

In [2]:
bsz = 3
seqlen= 4
num_experts = 5
input_dim = 6
output_dim = 2

In [3]:
x = torch.randn(3,4,6)

In [19]:
type_weights = torch.randn(bsz, seqlen, num_experts)

In [20]:
selected_experts = torch.ge(type_weights, 0).to(torch.float)

In [21]:
selected_experts

tensor([[[0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 1.],
         [1., 0., 1., 1., 1.],
         [1., 1., 1., 1., 0.]],

        [[1., 1., 0., 0., 0.],
         [0., 1., 0., 0., 1.],
         [1., 0., 0., 1., 1.],
         [1., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1.]]])

In [28]:
type_weights = type_weights * selected_experts

In [23]:
results = torch.zeros(x.shape[0], x.shape[1], output_dim, dtype=x.dtype, device=x.device) # [bsz, seqlen, output_dim]

In [29]:
type_weight = type_weights[:,:,0]
type_weight

tensor([[-0.0000, 2.0122, 0.1709, 1.5492],
        [0.8050, -0.0000, 0.1380, 0.5474],
        [-0.0000, -0.0000, -0.0000, 0.7196]])

In [33]:
# fake type weight, no expert was selected
type_weight = torch.zeros(bsz, seqlen)

In [34]:
torch.where(type_weight)

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [14]:
type_weight[batch_idx].shape

torch.Size([7])

In [44]:
results[batch_idx] += type_weight[batch_idx[0],batch_idx[1], None] * selected_x

In [35]:
# type_weight: [bsz, seqlen]

batch_idx = torch.where(type_weight)

selected_x = x[batch_idx] # [m, dim] ,m tokens selected for this expert
expert = nn.Linear(input_dim, output_dim)
selected_x = expert(selected_x)

if len(batch_idx[0])>0:
    results[batch_idx] += type_weight[batch_idx].unsqueeze(-1) * selected_x

In [37]:
selected_x

tensor([], size=(0, 2), grad_fn=<AddmmBackward0>)

In [13]:
results

tensor([[[-4.6016e-01,  3.6227e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 1.1564e-01,  1.9382e-01],
         [ 0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00],
         [ 4.3075e-03,  7.6672e-03],
         [ 0.0000e+00,  0.0000e+00],
         [ 8.4307e-02,  1.8493e-01]],

        [[ 1.8971e-03,  4.1265e-03],
         [-5.2414e-03, -1.3539e-01],
         [ 0.0000e+00,  0.0000e+00],
         [ 7.7153e-05, -1.7514e-01]]], grad_fn=<IndexPutBackward0>)

In [6]:
logits

tensor([[[ 0.5533,  0.2763,  1.5826, -0.3870, -0.3407],
         [-0.0127,  0.2232, -1.0327,  1.3578, -1.6212],
         [ 0.7360,  0.6163,  0.1449, -0.5394, -0.2685],
         [ 0.2271, -0.1391,  0.8285, -0.4191, -1.2136]],

        [[ 0.6697,  0.0855, -0.1071,  2.2308,  1.1990],
         [ 0.3835,  0.4575, -0.1208,  1.2266,  1.9493],
         [ 0.2710, -0.8836, -0.5274, -1.4447, -2.8189],
         [-1.3393,  0.9550, -1.6640,  0.9063, -0.0281]],

        [[ 1.0273, -0.1172,  1.4387,  0.6669, -1.0572],
         [-0.7939, -0.8781,  0.2611,  0.8649,  1.8302],
         [-0.4048, -1.8356, -0.1247, -1.9480, -0.9919],
         [ 2.4760, -1.2398, -1.9687, -0.4995,  1.0303]]])

In [7]:
logits = logits * selected_experts

In [8]:
expert0 = logits[:,:,0]

In [9]:
expert0

tensor([[0.5533, -0.0000, 0.7360, 0.2271],
        [0.6697, 0.3835, 0.2710, -0.0000],
        [1.0273, -0.0000, -0.0000, 2.4760]])

In [10]:
torch.where(expert0)

(tensor([0, 0, 0, 1, 1, 1, 2, 2]), tensor([0, 2, 3, 0, 1, 2, 0, 3]))

In [11]:
input = torch.randn(bsz, seqlen, 2)

In [14]:
selected_x = input[torch.where(expert0)]
selected_x

tensor([[-1.6389, -0.0037],
        [-1.5669,  0.9592],
        [ 0.9162, -0.4282],
        [-0.3991, -0.1021],
        [ 0.4370,  0.1516],
        [-2.1480,  0.1017],
        [ 1.1305, -1.4910],
        [ 0.9357, -1.7524]])

In [7]:
import torch

a= torch.rand(3,4,5)
torch.sum(a, (0,1))

tensor([6.7289, 6.5646, 5.6057, 6.8611, 7.6274])

In [8]:
torch.mean(a, (0,1))*12

tensor([6.7289, 6.5646, 5.6057, 6.8611, 7.6274])

In [9]:
b = torch.rand(3,4,5)

In [10]:
a += b

In [12]:
torch.sum(a, (0,1))

tensor([12.1682, 12.7050, 11.2053, 11.5993, 13.0622])

In [13]:
not {}

True

In [19]:
a = {0:2}

In [20]:
b = a 

In [22]:
a = {0:1}

In [23]:
a

{0: 1}

In [27]:
b

{0: 2}